In [ ]:
import numpy as np
import pandas as pd
import os
import pynetlogo
from tqdm import tqdm
import networkx as nx

In [ ]:
def getGiantComponentSize():
    world = netlogo.report("getAllPatchesColors")
    k_values = world[:, 2]
    mask = k_values == 11.5
    burned_world = world[mask]
    # Create a new NetworkX graph
    G = nx.Graph()
    # Add nodes (vertices) to the graph
    for element in burned_world:
        G.add_node(tuple([element[0], element[1]]))

    for node in G.nodes():
        (i, j) = node
        node1 = (i + 1, j)
        node2 = (i - 1, j)
        node3 = (i, j + 1)
        node4 = (i, j - 1)
        if node1 in G.nodes():
            G.add_edge(node, node1)
        if node2 in G.nodes():
            G.add_edge(node, node2)
        if node3 in G.nodes():
            G.add_edge(node, node3)
        if node4 in G.nodes():
            G.add_edge(node, node4)
    component_sizes = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
    return component_sizes[0]

In [ ]:
data = {
    'L/2': [],
    'density': [],
    'probability_of_spread': [],
    'south_wind_speed': [],
    'west_wind_speed': [],
    'big_jumps': [],
    'percentage_burned': [],
    'giant_component': []
}

df = pd.DataFrame(data)
print(df)

In [ ]:
# Initialize pynetlogo
netlogo = pynetlogo.NetLogoLink(
    gui=True,
    jvm_path="C:\\Program Files\\Java\\jdk-20\\bin\\server\\jvm.dll",  # Update with the correct path to your JDK's jvm.dll
)
# Load the "Fire Simple Extension 3" model from the NetLogo Model Library
netlogo.load_model("C:\\Program Files\\NetLogo 6.3.0\\models\\IABM Textbook\\chapter 3\\Fire Extensions\\Fire Simple Extension 3.nlogo")


In [ ]:
##### Add the following code to netlogo...
# to-report getAllPatchesColors
#   let color-list []
#   ask patches [
#     let patch-info (list pxcor pycor pcolor)
#     set color-list lput patch-info color-list
#   ]
#   report color-list
# end

In [ ]:
n = 3
list_of_l_2 = [450]
list_of_big_jump = ["false", "true"]
list_of_south_wind_speed = [-20, -10, 0, 10, 20]
list_of_west_wind_speed = [-20, -10, 0, 10, 20]
list_of_probability_of_spread = [20, 40, 60, 80, 100]
list_of_density = [10, 20, 30, 40, 50, 60, 70, 80, 90]

total_data = n * len(list_of_l_2) * len(list_of_big_jump) * len(list_of_south_wind_speed) * len(list_of_west_wind_speed) * len(list_of_density) * len(list_of_probability_of_spread)
pbar = tqdm(total=total_data, desc='Adding data', unit='data')

for i in range(n):
    for l_2 in list_of_l_2:
        netlogo.command("resize-world " + str(-l_2) + " " + str(l_2) + " " + str(-l_2) + " " + str(l_2))
        for big_jump in list_of_big_jump:
            for south_wind_speed in list_of_south_wind_speed:
                for west_wind_speed in list_of_west_wind_speed:
                    for probability_of_spread in list_of_probability_of_spread:
                        for density in list_of_density:
                            netlogo.command("set big-jumps? " + big_jump)
                            netlogo.command("set south-wind-speed " + str(south_wind_speed))
                            netlogo.command("set west-wind-speed " + str(west_wind_speed))
                            netlogo.command("set probability-of-spread " + str(probability_of_spread))
                            netlogo.command("set density " + str(density))
                            netlogo.command("setup")
                            old_ticks = netlogo.report("ticks")
                            while (True):
                                netlogo.command("go")
                                current_ticks = netlogo.report("ticks")
                                if (old_ticks == current_ticks):
                                    break
                                else:
                                    old_ticks = current_ticks
                            percentage_burned = netlogo.report("((count patches with [shade-of? pcolor red]) / initial-trees) * 100")
                            giant_component = getGiantComponentSize()
                            new_row = {
                                'L/2': l_2,
                                'density': density,
                                'probability_of_spread': probability_of_spread,
                                'south_wind_speed': south_wind_speed,
                                'west_wind_speed': west_wind_speed,
                                'big_jumps': big_jump,
                                'percentage_burned': percentage_burned,
                                'giant_component': giant_component
                            }
                            # Append the new row to the DataFrame
                            df = df.append(new_row, ignore_index=True)
                            pbar.update(1)

In [ ]:
df.to_csv("giant_main.csv")